In [67]:
#import packages
import pandas as pd
import datetime as dt
import numpy as np

import os


In [68]:
#missing datas / map empty cells
#=> terminal : pip install missingno
!python3 -m pip install missingno
import missingno



ModuleNotFoundError: No module named 'missingno'

In [65]:
#import file and first infos
list_clics = pd.read_csv("C:/Users/BGE/Google Drive/GIT/CSV/[csv] Leads_Analysis/list_cleaned_02_28.csv")
list_clics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22385 entries, 0 to 22384
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Entrepreneur          22385 non-null  object
 1   Email                 22385 non-null  object
 2   Partenaire            22385 non-null  object
 3   Catégorie partenaire  22385 non-null  object
 4   Conseiller            22385 non-null  object
 5   Antenne               22385 non-null  object
 6   date_without_time     22385 non-null  object
dtypes: object(7)
memory usage: 1.2+ MB


In [66]:
# discover structure of dataframe
list_clics.head()


,Entrepreneur,Email,Partenaire,Catégorie partenaire,Conseiller,Antenne,date_without_time
0,ABDELKADER ADDA-BOUSEKRANE,aababdelkader@gmail.com,BDL,2- Comptabilité,Julien BERRUEZO,Valenciennes,2022-09-26
1,ABDELKADER ADDA-BOUSEKRANE,aababdelkader@gmail.com,TERRE D'ENTREPRENEURS,2- Comptabilité,Julien BERRUEZO,Valenciennes,2022-09-26
2,ABDELKADER ADDA-BOUSEKRANE,aababdelkader@gmail.com,Valoxy Denain,2- Comptabilité,Julien BERRUEZO,Valenciennes,2022-09-26
3,ABDOULHANYOU SAROUMA,saroumanailane@gmail.com,Allianz Avesnois,4 - Assurance,Fanny FORTIN,Maubeuge,2022-01-11
4,ABDOULHANYOU SAROUMA,saroumanailane@gmail.com,Cabinet d'expertise comptable Grosso Maubeuge,2- Comptabilité,Fanny FORTIN,Maubeuge,2022-01-11


In [ ]:
#nb of lines
list_clics.count()

Entrepreneur            22385
Email                   22385
Partenaire              22385
Catégorie partenaire    22385
Conseiller              22385
Antenne                 22385
date_without_time       22385
dtype: int64

In [ ]:
# column structures
list_clics.columns

Index(['Entrepreneur', 'Email', 'Partenaire', 'Catégorie partenaire',
       'Conseiller', 'Antenne', 'date_without_time'],
      dtype='object')

In [ ]:
# duplicates
list_unduplicated = list_clics.sort_values(by=["Entrepreneur", "Partenaire"])
list_unduplicated.drop_duplicates()
list_unduplicated.count()

Entrepreneur            22385
Email                   22385
Partenaire              22385
Catégorie partenaire    22385
Conseiller              22385
Antenne                 22385
date_without_time       22385
dtype: int64

In [ ]:
#truncate without hours + datetime format
#Date_rel_dt = pd.to_datetime(list_unduplicated['Date mise en relation'])
#Date_rel_dt.dt.date

list_unduplicated = list_unduplicated.assign(date_without_time = lambda x: pd.to_datetime(x['Date mise en relation']).dt.date)
type(list_unduplicated)


KeyError: 'Date mise en relation'

In [ ]:
list_unduplicated.head()

In [ ]:
#map empty/na cells
mno.matrix(list_unduplicated)

In [ ]:
#check if any null data
list_unduplicated.isnull().sum()


PARTNERS LIST TABLES

In [ ]:
#create partners table
df_partners = list_unduplicated[~list_unduplicated.duplicated(subset=['Partenaire'])].copy()
df_partners = df_partners[ ['Partenaire', 'Contact partenaire'] ]
df_partners['Partenaire'].str.strip()
df_partners.sort_values(by='Partenaire')
#df_partners['Partenaire'] = pd.Series(df_partners['Partenaire'])

#create index
erase_space = lambda x : x.replace(' ', '')
strip_index = lambda x : x.str.strip()
# df_partners['idPartners'] = df_partners['Partenaire'].apply(erase_space, strip_index)
df_partners.sort_values(by='Partenaire', inplace=True)

df_partners.loc[(df_partners['Partenaire'].str.contains('Groupama'))]
#df_partners[groupama]

KeyError: "['Contact partenaire'] not in index"

In [ ]:
#create partners table
list_unduplicated.sort_values(by='Date mise en relation')
df_partners = list_unduplicated.groupby(['Partenaire']).first()

df_partners

KeyError: 'Date mise en relation'

In [ ]:
#replace empty cells
#def str_or_not(x):
    #if x == '':
        #if isinstance(x, str):
        #x.replace('', np.nan)
    #"else: x

#list_unduplicated_unblanked = list_unduplicated.apply(lambda x: x.str.strip().replace('', np.nan))
list_unduplicated_unblanked = list_unduplicated.replace('', np.nan)
#mno.matrix(list_unduplicated_unblanked)

#list_unduplicated_unblanked.isnull().sum()
list_unduplicated_unblanked.isna().sum()
# type(list_unduplicated_unblanked)



In [ ]:
#fillna blank cells
list_unduplicated_unblanked['Contact partenaire'].fillna('inconnu@adfinir.com', inplace=True)
list_unduplicated_unblanked['Telephone'].fillna('0600000000', inplace=True)
list_unduplicated_unblanked.isna().sum()



In [ ]:
list_unduplicated_unblanked.head()

In [ ]:
list_unduplicated_unblanked['Entrepreneur'] = list_unduplicated_unblanked['Entrepreneur'].str.lower()

NameError: name 'list_unduplicated_unblanked' is not defined

In [ ]:
list_duplicated_unblanked=list_unduplicated_unblanked.duplicated(subset=['Entrepreneur', 'Email', 'Telephone', 'Partenaire', 'Catégorie partenaire', 'Conseiller','Antenne', 'Date mise en relation', 'date_without_time'])
list_unduplicated_unblanked = list_unduplicated_unblanked[~list_duplicated_unblanked]
list_unduplicated_unblanked 

In [ ]:
#erase useless columns
list_cleaned = list_unduplicated_unblanked.drop(columns=['Telephone', 'BGE Club', 'Contact partenaire', 'Date mise en relation', 'A été contacté', 'A eu un échange', 'Note'] )
list_cleaned.reset_index()

In [ ]:
#low and clean 'Entrepreneur' column

from unidecode import unidecode

lower_index = lambda x : x.lower()
erase_indent = lambda x : x.replace('-', '')
strip_index = lambda x : x.strip()
accents_strip = lambda x : unidecode(x)

list_cleaned['Entrepreneur'] = list_cleaned['Entrepreneur'].apply(strip_index).apply(erase_indent).apply(lower_index).apply(accents_strip)

# # Print the DataFrame to verify the changes
list_cleaned

In [ ]:
#create a csv file
list_cleaned.to_csv('list_cleaned_02_28.csv', index=False)